In [70]:
import warnings
warnings.simplefilter('ignore')

# imports

import pandas as pd
import numpy as np

In [71]:
#read csv files in

df = pd.read_csv("Result/NBA_data_all_80-19.csv")
df2020 = pd.read_csv("Result/NBA_data_all_19-20.csv")

In [72]:
df["playoffs_y_n"] = df["Team"].map(lambda x: 1 if "*" in x else 0)
df["Team"] = df["Team"].str.replace("*", "")
df = df.dropna()

df2020["playoffs_y_n"] = df2020["Team"].map(lambda x: 1 if "*" in x else 0)
df2020["Team"] = df2020["Team"].str.replace("*", "")
df2020 = df2020.dropna()
prediction_df = df2020
prediction_df = prediction_df.drop(columns=["RK","Team","Year","Conf","playoffs_y_n"])

In [73]:
model_df = df.loc[(df["Year"] != 2017) & (df["Year"] != 2018) & (df["Year"] != 2019) & (df["Year"] != 2020)]
recent_years_df = df.loc[(df["Year"] == 2017) | (df["Year"] == 2018) | (df["Year"] == 2019) | (df["Year"] == 2020)]

# train_df = train_df.drop(columns=["RK","Team","Year","Conf"])
recent_years_test_df = recent_years_df.drop(columns=["RK","Team","Year","Conf","playoffs_y_n"])

In [74]:
from sklearn.model_selection import train_test_split

y = model_df["playoffs_y_n"]
X = model_df.drop(columns=["RK","Team","Year","Conf","playoffs_y_n"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [75]:
# y_test = test_df["playoffs_y_n"]
# X_test = test_df.drop(columns=["playoffs_y_n"])

# y_train = train_df["playoffs_y_n"]
# X_train = train_df.drop(columns=["playoffs_y_n"])

In [76]:
#from sklearn.preprocessing import MinMaxScaler
#X_scaler = MinMaxScaler().fit(X_train)

#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)

In [77]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [78]:
print(f"Training Data Score: {model.score(X_train, y_train)}")
print(f"Testing Data Score: {model.score(X_test, y_test)}")

Training Data Score: 0.9315789473684211
Testing Data Score: 0.9094488188976378


In [79]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'penalty': ["l1", "l2"]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [80]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] C=1, penalty=l1 .................................................
[CV] ....................... C=1, penalty=l1, score=nan, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] ....................... C=1, penalty=l1, score=nan, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] ....................... C=1, penalty=l1, score=nan, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] ....................... C=1, penalty=l1, score=nan, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] ....................... C=1, penalty=l1, score=nan, total=   0.0s
[CV] C=1, penalty=l2 .................................................
[CV] ..................... C=1, penalty=l2, score=0.941, total=   0.0s
[CV] C=1, penalty=l2 .................................................
[CV] ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ..................... C=5, penalty=l2, score=0.947, total=   0.0s
[CV] C=5, penalty=l2 .................................................
[CV] ..................... C=5, penalty=l2, score=0.934, total=   0.1s
[CV] C=5, penalty=l2 .................................................
[CV] ..................... C=5, penalty=l2, score=0.908, total=   0.0s
[CV] C=5, penalty=l2 .................................................
[CV] ..................... C=5, penalty=l2, score=0.868, total=   0.0s
[CV] C=5, penalty=l2 .................................................
[CV] ..................... C=5, penalty=l2, score=0.941, total=   0.0s
[CV] C=10, penalty=l1 ................................................
[CV] ...................... C=10, penalty=l1, score=nan, total=   0.0s
[CV] C=10, penalty=l1 ................................................
[CV] ...................... C=10, penalty=l1, score=nan, total=   0.0s
[CV] C=10, penalty=l1 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.6s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10], 'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [81]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'penalty': 'l2'}
0.9236842105263158


In [82]:
predictions = model.predict_proba(recent_years_test_df)
print("Classes are either 0 (did not qualify for playoffs) or 1 (qualified for playoffs)")
print(f"The new point was classified as: {predictions}")

Classes are either 0 (did not qualify for playoffs) or 1 (qualified for playoffs)
The new point was classified as: [[2.95024904e-07 9.99999705e-01]
 [3.57392611e-04 9.99642607e-01]
 [4.89531011e-01 5.10468989e-01]
 [1.74666304e-02 9.82533370e-01]
 [1.51823349e-02 9.84817665e-01]
 [4.51009401e-03 9.95489906e-01]
 [4.56293510e-04 9.99543706e-01]
 [6.80378666e-01 3.19621334e-01]
 [9.99784857e-01 2.15142735e-04]
 [1.78255737e-02 9.82174426e-01]
 [4.36054298e-02 9.56394570e-01]
 [9.99925320e-01 7.46799405e-05]
 [9.77917897e-01 2.20821026e-02]
 [3.53312104e-05 9.99964669e-01]
 [2.66085665e-01 7.33914335e-01]
 [5.84349836e-01 4.15650164e-01]
 [9.99542720e-01 4.57280358e-04]
 [9.14357184e-01 8.56428162e-02]
 [9.95669093e-01 4.33090741e-03]
 [2.02418736e-01 7.97581264e-01]
 [7.71612265e-01 2.28387735e-01]
 [8.17164660e-02 9.18283534e-01]
 [2.59895992e-01 7.40104008e-01]
 [9.76651709e-01 2.33482911e-02]
 [9.93743556e-01 6.25644447e-03]
 [9.19150864e-01 8.08491362e-02]
 [9.94138500e-01 5.86150015

In [83]:
test = predictions[:,1].tolist()

In [84]:
#df2019 = df2018.dropna()
recent_years_df["playoff prediction"] = test

In [85]:
recent_years_df

,RK,Team,Year,G,W,L,Conf,MP,FG,FGA,...,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,playoffs_y_n,playoff prediction
1014,1,Golden State Warriors,2017,82,67,15,West,241.2,43.1,87.1,...,9.4,35.0,30.4,9.6,6.8,14.8,19.3,115.9,1,1.000000
1015,2,Houston Rockets,2017,82,55,27,West,241.2,40.3,87.2,...,10.9,33.5,25.2,8.2,4.3,15.1,19.9,115.3,1,0.999643
1016,3,Denver Nuggets,2017,82,40,42,West,240.9,41.2,87.7,...,11.8,34.6,25.3,6.9,3.9,15.0,19.1,111.7,0,0.510469
1017,4,Cleveland Cavaliers,2017,82,51,31,East,242.4,39.9,84.9,...,9.3,34.4,22.7,6.6,4.0,13.7,18.1,110.3,1,0.982533
1018,5,Washington Wizards,2017,82,49,33,East,242.1,41.3,87.0,...,10.3,32.6,23.9,8.5,4.1,14.2,21.3,109.2,1,0.984818
1019,6,Los Angeles Clippers,2017,82,51,31,West,240.9,39.5,83.2,...,9.0,34.0,22.5,7.5,4.2,13.0,19.8,108.7,1,0.995490
1020,7,Boston Celtics,2017,82,53,29,East,240.9,38.6,85.1,...,9.1,32.9,25.2,7.5,4.1,13.3,20.6,108.0,1,0.999544
1021,8,Portland Trail Blazers,2017,82,41,41,West,243.0,39.5,86.1,...,10.1,33.5,21.1,7.0,5.0,13.7,21.2,107.9,1,0.319621
1022,9,Phoenix Suns,2017,82,24,58,West,241.8,39.9,88.5,...,11.9,33.1,19.6,8.2,4.9,15.4,24.8,107.7,0,0.000215
1023,10,Toronto Raptors,2017,82,51,31,East,241.2,39.2,84.4,...,10.6,32.6,18.5,8.3,4.9,12.7,20.8,106.9,1,0.982174


In [ ]:
predictions = model.predict_proba(prediction_df)
print("Classes are either 0 (did not qualify for playoffs) or 1 (qualified for playoffs)")
print(f"The new point was classified as: {predictions}")

In [13]:
predictions[:,1]

array([7.90426934e-01, 9.99889172e-01, 9.00265657e-01, 5.40532180e-03,
       7.66852086e-04, 4.71622719e-02, 9.92221937e-01, 1.28981373e-02,
       3.56336915e-01, 7.30495866e-02, 9.84598960e-01, 1.05449883e-03,
       9.81504066e-01, 9.98633988e-01, 9.99538267e-01, 9.90465562e-01,
       4.29158747e-02, 9.47472762e-01, 9.16018109e-01, 8.99334087e-01,
       9.52083537e-01, 1.23946522e-01, 6.24460134e-04, 8.64980244e-04,
       2.79106898e-02, 1.97166318e-04, 3.10989554e-03, 2.10911023e-01,
       4.63469345e-04, 3.94482008e-03])

In [14]:
test = predictions[:,1].tolist()

In [15]:
#df2019 = df2018.dropna()
df2020["playoff prediction"] = test

In [16]:
df2020 = df2020.sort_values(by=["playoff prediction"],ascending=False)


In [22]:
#df2020 = df2020.reset_index()
df2020 = df2020.drop(columns=["level_0"])

In [23]:
df2020

,RK,Team,Year,G,W,L,Conf,MP,FG,FGA,...,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,playoffs_y_n,playoff prediction
0,2,Milwaukee Bucks,2020,65,53,12,East,240.8,43.5,91.2,...,9.5,42.2,25.9,7.4,6.0,14.9,19.2,118.6,1,0.999889
1,15,Los Angeles Lakers,2020,63,49,14,West,240.8,42.9,88.6,...,10.6,35.5,25.9,8.6,6.8,15.1,20.6,114.3,1,0.999538
2,14,Toronto Raptors,2020,64,46,18,East,241.6,40.6,88.5,...,9.7,35.5,25.4,8.8,4.9,14.4,21.5,113.0,1,0.998634
3,7,Los Angeles Clippers,2020,64,44,20,West,241.2,41.6,89.7,...,11.0,37.0,23.8,7.1,5.0,14.8,22.0,116.2,0,0.992222
4,16,Denver Nuggets,2020,65,43,22,West,242.3,41.8,88.9,...,10.8,33.5,26.5,8.1,4.6,13.7,20.0,110.4,0,0.990466
5,11,Miami Heat,2020,65,41,24,East,243.5,39.6,84.4,...,8.5,36.0,26.0,7.4,4.5,14.9,20.4,112.2,0,0.984599
6,13,Boston Celtics,2020,64,43,21,East,242.0,41.2,89.6,...,10.7,35.3,22.8,8.3,5.6,13.6,21.4,113.0,1,0.981504
7,21,Oklahoma City Thunder,2020,64,40,24,West,241.6,40.3,85.1,...,8.1,34.6,21.9,7.6,5.0,13.5,18.8,110.8,0,0.952084
8,18,Philadelphia 76ers,2020,65,39,26,East,241.2,40.8,87.7,...,10.4,35.1,25.9,8.2,5.4,14.2,20.6,109.6,0,0.947473
9,19,Indiana Pacers,2020,65,39,26,East,241.5,42.2,88.4,...,8.8,34.0,25.9,7.2,5.1,13.1,19.6,109.3,0,0.916018
